In [5]:
import sys
a = sys.stdout
reload(sys)
sys.setdefaultencoding("utf-8")
sys.stdout = a

import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_sentence(sentence):
    res = []
    for word, pos in pos_tag(word_tokenize(sentence)):
#        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        wordnet_pos = get_wordnet_pos(pos)
#        print wordnet_pos
        if wordnet_pos=='v' or wordnet_pos=='a'or wordnet_pos=='n':
#        if wordnet_pos=='v':
            res.append(lemmatizer.lemmatize(word, pos=wordnet_pos))
#        else:
#            print wordnet_pos
    return res


for index in range(10000):  
    data=[]
    for line in open("/Users/leafy/Downloads/data/descriptions_test/" + str(index) + ".txt"):
        l = line.strip('\n')
        data.append(l)
    data = np.array(data) 
    
    data = np.char.lower(data)
    
    for i in range(len(data)):
    #lemmatize
        data[i]=" ".join(lemmatize_sentence(data[i]))    
    #Strip the stop words

    for i in range(len(data)):
        s = []
        for word in data[i].split():
            try:
                if word not in stopwords.words('english'):
                    s.append(word)
#                 else:
#                     print(word)
            except:
                pass
        data[i] = " ".join(s)        
       
        #Strip punctuation
    import string
    for j in range(len(data)):
        data[j] = data[j].translate(None, string.punctuation)
    number='1234567890'
    for j in range(len(data)):
        data[j] = data[j].translate(None, number) 
#         print(data[i])
    f = open('/Users/leafy/anaconda/final_project/CS5785-Fall-2017-Final-Exam/features_test_prepossesing_verb&adj&noun/' + str(index) + ".txt", 'w+')
    for i in range(len(data)):
        for s in data[i].split(" "):
            f.write(s + " ")
    f.close()


IOError: [Errno 2] No such file or directory: '/Users/leafy/Downloads/data/descriptions_test/2000.txt'

In [3]:
# First find keyword based matcing strategy.

# Need to start reading form here - http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7477706

from os.path import join

from glob import glob

from collections import defaultdict

from sklearn.preprocessing import MultiLabelBinarizer

import csv

import re

data_path='data'
tags_test='tags_test'
file_categories = defaultdict(int)
file_tags = defaultdict(int)
for file_name in glob(join(data_path, tags_test, "*.txt")):
    for line in open(file_name).readlines():
        file_tags[line.strip().split(":")[0]] += 1
        file_categories[line.strip().split(":")[1]] += 1

labels = list(set(list(file_categories.keys()) + list(file_tags.keys())))

mlb = MultiLabelBinarizer()
mlb.fit([labels])

pattern = "\d.*txt"

rows = []
for file_name in sorted(glob(join(data_path, tags_test, "*.txt"))):
    image_set = set()
    for line in open(file_name).readlines():
        image_set = image_set.union(set([line.strip().split(":")[0], line.strip().split(":")[1]]))

    rows.append([int(re.findall(pattern=pattern, string=file_name)[0].strip("\.txt"))] + list(mlb.transform([image_set])[0]))
rows = sorted(rows, key=lambda x: x[0])

csv.writer(open('processed_tags_test.csv', "w")).writerows([["Name"] + list(mlb.classes_)] + rows)

In [186]:
import pandas as pd
tag_test = pd.read_csv("processed_tags_test.csv")
tag_test = tag_test.columns[1:]
tag_dict = dict()
index = 0
for word in tag_test:
    if word not in tag_dict:
        tag_dict[word] = index
        index += 1
#print tag_dict
#print tag_test
tag_feat=[]
with open('processed_tags_test.csv') as csvfile:
    lines = csv.reader(csvfile)
    for index,line in enumerate (lines):
        if index!=0:
            tag_feat.append(line[1:])
tag_feat=np.array(tag_feat, dtype= float )
data_test=[]
for index in range(2000):  
    data=[]
    for line in open("features_test_prepossesing/" + str(index) + ".txt"):
        l = line.strip('\n')
        data.append(l)
#        data = np.array(data)
    data_test.append(data)
#data_test=np.array(data_test)

test_feat = []
word_count = len(tag_dict)
for sentence in data_test:
    cur=[]
    for tag in tag_test:
        cur.append(sentence[0].count(tag))
    test_feat.append(cur)
    
    
#for sentence in data_test:
    
#    cur = [0] * word_count
#    for word in sentence[0].split():
#        if word in tag_dict:
#            cur[tag_dict[word]] += 1      
#    test_feat.append(cur)
test_feat = np.array(test_feat, dtype= float )
#print test_feat
#test_feat[1].shape
#for word in tag_dict:
#    print word.split()
print tag_feat

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  1.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [195]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(norm='l2',use_idf=True,smooth_idf=False,sublinear_tf=False)
test_feat = transformer.fit_transform(test_feat).toarray()
tag_feat= transformer.fit_transform(tag_feat).toarray()
print tag_feat

[[ 0.13643343  0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.12890716 ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.11821773  0.          0.        ]
 [ 0.          0.98986007  0.         ...,  0.1420459   0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]


In [192]:
import csv

from scipy.spatial.distance import cdist
simi=cdist(test_feat,tag_feat,metric='cosine')
rows=[['Descritpion_ID', 'Top_20_Image_IDs']]
for i in range (len(test_feat)):
    buffer1=[]
    buffer1=simi[i].argsort()[:20]
    rows.append([str(i) + ".txt", " ".join([str(value) + ".jpg" for value in buffer1])])
csv.writer(open("submission_try.csv", "w")).writerows(rows)

1.0


kate
build
cat
